In [ ]:
import torch
import matplotlib.pyplot as plt

print(torch.version.cuda)
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
print('Device:', device)

12.1
True
Device: cuda


In [ ]:
from torchvision.transforms import v2
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader, random_split

transform = v2.Compose([
    v2.Resize((128, 128)),
    v2.Grayscale(num_output_channels=1),
    v2.RandomRotation(10),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485], std=[0.229])
])

full_dataset = datasets.CIFAR10(root='./data_Cifar10', train=True, download=True,
                                     transform=transform,
                                     target_transform=v2.Lambda(lambda y: torch.nn.functional.one_hot(torch.tensor(y), num_classes=10).float() * 0.9 + 0.1/10)
                                     )


train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


# Load the test dataset
test_dataset = datasets.CIFAR10(root='./data_Cifar10', train=False, download=True,
                                     transform=transform,
                                     target_transform=v2.Lambda(lambda y: torch.nn.functional.one_hot(torch.tensor(y), num_classes=10).float() * 0.9 + 0.1/10)
                                     )

In [ ]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
print(f"Number of classes: {len(train_dataset.dataset.classes)}")

class_mapping_idx_to_name = dict(enumerate(train_dataset.dataset.classes))
print(f"Class mapping: {class_mapping_idx_to_name}")

class_mapping_name_to_idx = train_dataset.dataset.class_to_idx
print(f"Class mapping: {class_mapping_name_to_idx}")

In [6]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Cifar10CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.CELU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.Dropout2d(p=0.1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.Dropout2d(p=0.25),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),

            nn.Linear(in_features= 16 * 16 * 64, out_features=256),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=num_classes)

        )
    def forward(self, x):
        logits = self.model(x)
        return F.softmax(logits, dim=1)

In [7]:
input_size = 128 * 128
num_classes = 10

model = Cifar10CNN(num_classes)

model.to(device)

Cifar10CNN(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): CELU(alpha=1.0)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Dropout2d(p=0.1, inplace=False)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): Dropout2d(p=0.25, inplace=False)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Flatten(start_dim=1, end_dim=-1)
    (12): Linear(in_features=16384, out_features=256, bias=True)
    (13): Dropout(p=0.5, inplace=False)
    (14): ReLU()
    (15): Linear(in_features=128, out_features=10, bias=True)
  )
)